# CNN with 1D kernels and striding

In [4]:
# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.__version__

'2.2.0'

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!apt install fluidsynth
!pip install midi2audio
!pip install pretty_midi

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (1.1.9-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Input, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback
import numpy as np
import pylab as plt

import sys
sys.path.insert(1, '/content/gdrive/My Drive/Music Gen/import/')
import models
import trainFac
import midiUtils

## NN Architecture

In [2]:
class Model_CNN_Stride(models.Model_1Dout):

    def __init__(self, input_length):
        super().__init__(input_length)
        self.createModel()

    def createModel(self):
        model = Sequential()
        model.add(Input(shape=(self.input_length, 48, 1)))

        model.add(Conv2D(16, (1, 5), activation='relu'))
        model.add(Conv2D(16, (3, 1), activation='relu', strides=(2, 1), padding='same'))

        model.add(Conv2D(16, (1, 5), activation='relu'))
        model.add(Conv2D(16, (3, 1), activation='relu', strides=(2, 1), padding='same'))

        model.add(Conv2D(32, (5, 5), activation='relu'))

        model.add(Flatten())
        model.add(Dense(48, activation='sigmoid'))
        # save to self
        self.model = model


modelCompound = Model_CNN_Stride(120)

# modelCompound.model.load_weights('/content/gdrive/My Drive/Music Gen/networks/cnn_stride1D_kernel1D/train1/checkpoints/cnn-20-accuracy0.504.hdf5')
modelCompound.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 44, 16)       96        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 44, 16)        784       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 40, 16)        1296      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 40, 16)        784       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 36, 32)        12832     
_________________________________________________________________
flatten (Flatten)            (None, 29952)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1

## Training

In [3]:
paths = trainFac.Paths(
    rootDir = '/content/gdrive/My Drive/Music Gen',
    trainPath = 'train_batches/train_7840_chorus_npz_500',
    testPath  = 'test_batches/test_800_chorus_npz_100',
    testSongPath = 'test_generation/test_songs',
    testGenerateSavePath = 'networks/cnn_stride1D_kernel1D/train1/generate',
    checkpointBasePath = 'networks/cnn_stride1D_kernel1D/train1/checkpoints',
    checkpointName='cnn-{epoch:02d}-accuracy{val_accuracy:.3f}.hdf5')
# create paths if they dont exist
paths.createSavePaths()

trainFacility = trainFac.TrainFacility(modelCompound, paths)
trainFacility.compile()

trainFacility.modelCompound.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 120, 44, 16)       96        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 44, 16)        784       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 40, 16)        1296      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 40, 16)        784       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 36, 32)        12832     
_________________________________________________________________
flatten (Flatten)            (None, 29952)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1

In [0]:
history = trainFacility.fit(epochs=20, steps_per_epoch=10, validation_steps=10)

In [0]:
import json
with open('/content/gdrive/My Drive/Music Gen/networks/cnn_stride1D_kernel1D/train1/history.json', 'w') as fp:
    json.dump(history.history, fp, indent=4)

## Experiments

In [0]:
net_name = 'CNN Stride'
save_path = '/content/gdrive/My Drive/Music Gen/results/cnn-stride/'

In [9]:
song_name = "Can't Stop"
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/cant_stop.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [10]:
song_name = 'Hells Bells'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/hells_bells.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Layla'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/layla.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.2)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Nothing Else Matters'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/nothing_else_matters.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Smells Like Teen Spirit'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/smells_like_teen_spirit.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.2)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Under The Bridge'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/under_the_bridge.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.2)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven 2'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven_2.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.10)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Wonderwall'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/wonderwall.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.2)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.